In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium_stealth import stealth
service = ChromeService(executable_path=ChromeDriverManager().install())

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
import os
import pandas as pd

/Users/katieaebi/Desktop/Data Science major/Senior Portfolio/NYT Indep. Study/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
data = {}
for i, category in enumerate(os.listdir(os.path.join('.', '2023_bestsellers'))): 
    data[category[:-4]] = pd.read_csv(os.path.join('.', '2023_bestsellers', category))

In [10]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)
driver.get('https://www.goodreads.com/')
wait.until(lambda d: driver.execute_script("return document.readyState") == "complete")
assert 'Goodreads' in driver.title
signin_button = driver.find_element(By.XPATH, '//*[@id="signIn"]/div/div/a')
signin_button.click()

In [11]:
assert 'Sign in' in driver.title
email_button = driver.find_element(By.CLASS_NAME, "authPortalSignInButton")
email_button.click()

In [12]:
email_input = driver.find_element(By.XPATH, '//*[@id="ap_email"]')
password_input = driver.find_element(By.XPATH, '//*[@id="ap_password"]')
email_input.send_keys('______')
password_input.send_keys('_______')
driver.find_element(By.XPATH, '//*[@id="signInSubmit"]').click()

In [ ]:
############# testing navigation for loop: searching ###################
search_input = driver.find_element(By.CSS_SELECTOR, "input.searchBox__input.searchBox__input--navbar[placeholder='Search books'][aria-label='Search books']")
search_input.send_keys('9780525560654')
driver.find_element(By.CSS_SELECTOR, "button.searchBox__icon--magnifyingGlass.gr-iconButton.searchBox__icon.searchBox__icon--navbar[aria-label='Search']").click()

In [ ]:
############# testing navigation for loop: genres ###################
genres_container = driver.find_element(By.CSS_SELECTOR, "ul.CollapsableList")
genres_elements = genres_container.find_elements(By.CSS_SELECTOR, ".Button__labelItem")
genre_labels = []
for i, element in enumerate(genres_elements):
    if element.text != "...more":
        genre_labels.append(element.text)
print(genre_labels)

In [ ]:
############# testing navigation for loop: descriptions ###################
driver.find_element(By.CSS_SELECTOR, '.Button[aria-label="Tap to show more book description"]').click() #click show more
description = driver.find_element(By.CSS_SELECTOR, "span.Formatted") # find description
print(description.text)

In [ ]:
print(data['category']['hardcover-fiction_2023']['author'][0])

In [ ]:
############# testing navigation for loop: author-bios ###################
author_name = data['category']['hardcover-fiction_2023']['author'][0]
author_page = driver.find_element(By.LINK_TEXT, author_name).get_property('href')
driver.get(author_page)

In [ ]:
############# testing navigation for loop: author-bios ###################
born_element = driver.find_element(By.XPATH, "//div[@class='dataTitle' and text()='Born']")
print(born_element.get_property('nextSibling')['data'])
print(born_element.get_property('textContent'))

In [ ]:
driver.back()

In [ ]:
isbn_genre_data = {}
wait = WebDriverWait(driver, 10)
for i, category in enumerate(data.keys()):
    current_list = data[category]
    print(category)
    for j, ISBN in enumerate(current_list['primary_isbn13'].unique()): #loop through unique ISBNs for processing efficiency
        print(ISBN)
        search_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input.searchBox__input.searchBox__input--navbar[placeholder='Search books'][aria-label='Search books']")))
        search_input.send_keys(str(ISBN)) #search for book by ISBN
        try:
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.searchBox__icon--magnifyingGlass.gr-iconButton.searchBox__icon.searchBox__icon--navbar[aria-label='Search']"))).click() #click search button 
        except StaleElementReferenceException: 
            search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.searchBox__icon--magnifyingGlass.gr-iconButton.searchBox__icon.searchBox__icon--navbar[aria-label='Search']")))
            search_button.click()
        
        wait.until(lambda d: driver.execute_script("return document.readyState") == "complete")
        
        if len(data[category]['description'][data[category]['description'].isna() == True]) > 0:
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".Button[aria-label='Tap to show more book description']"))).click() #click show more
            description = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.Formatted"))) # find description
            #assigning the description everywhere this ISBN appears on the list 
            data['category'][category].loc[data['category'][category]['primary_isbn13'] == ISBN, 'description'] = description.text 
            print('got description!')
        try:
            genres_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul.CollapsableList")))
            genres_elements = wait.until(lambda d: genres_container.find_elements(By.CSS_SELECTOR, ".Button__labelItem"))       
            genre_labels = []
            for e, element in enumerate(genres_elements):
                if element.text != "...more":
                    genre_labels.append(element.text)
            isbn_genre_data[ISBN] = genre_labels
            print('got genres!')
        except StaleElementReferenceException:
            
        
        driver.back()
        wait.until(lambda d: driver.execute_script("return document.readyState") == "complete")
        assert 'Goodreads' in driver.title
            

In [ ]:
driver.quit()